In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import glob

In [2]:
from src.data_loader import load_dataset

/Users/lauraboggia/VSCode_projects/iTransformerAD/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def consecutive_nonzero_lengths(arr):
    lengths = []
    current_length = 0
    for val in arr:
        if val != 0:
            current_length += 1
        else:
            if current_length > 0:
                lengths.append(current_length)
                current_length = 0
    if current_length > 0:
        lengths.append(current_length)
    return lengths

In [4]:
train_loader, test_loader, labels, ts_lengths, enc_feats = load_dataset('UCR')

print(train_loader.dataset.data.shape, test_loader.dataset.data.shape)
true_labels = (np.sum(labels, axis=1) >= 1) + 0
print(true_labels.shape)
print(np.sum(true_labels), f'{np.sum(true_labels) / len(true_labels)*100:.2f} %')

anomaly_lens = consecutive_nonzero_lengths(true_labels)
print(anomaly_lens)
print(np.mean(anomaly_lens), np.max(anomaly_lens), np.min(anomaly_lens))

training set shape: (1600, 1)
test set shape: (5900, 1)
labels shape: (5900, 1)
ts_lengths 0: 1600
ts_lengths 1: 5900
(1600, 1) (5900, 1)
(5900,)
111 1.88 %
[111]
111.0 111 111


In [5]:
def load_dataset2(dataset, feats=-1, less=False, enc=False):
	folder = os.path.join('processed', dataset)
	if not os.path.exists(folder):
		raise Exception('Processed Data not found.')
	loader = []
	ts_lengths = []
	enc_feats = 0

	for file in ['train', 'test', 'labels']:
		if 'ATLAS_DQM' in dataset and file != 'train':
			# file = f'{file}_cosmicCalo'
			# file = f'{file}_hardProbes'
			file = f'{file}_pumpNoise'
			# file = f'{file}_hvononNominal'
		paths = glob.glob(os.path.join(folder, f'*{file}*.npy'))
		paths = sorted(paths)  # sort paths to ensure correct order, otherwise labels & test files are mismatched

		loader.append(np.concatenate([np.load(p) for p in paths]))
		ts_lengths.append([np.load(p).shape[0] for p in paths])
	
	if feats > 0:  # reduce number of features
		print(f'data set has {loader[0].shape[1]} features, only using {feats}')
		for i in range(2):
			max_feats = feats + enc_feats
			loader[i] = loader[i][:,:max_feats]
	
	train_loader = loader[0]
	test_loader = loader[1]
	labels = loader[2]
	
	# if labels are one dimensional, add axis
	if len(labels.shape) == 1:
		labels = labels[:, np.newaxis]
	if labels.shape[1] == 1: # if labels are 1D, repeat them for each feature to have 2D labels
		labels = np.repeat(labels, loader[0].shape[1], axis=1)
		
	print('training set shape:', train_loader.shape)
	print('test set shape:', test_loader.shape)
	print('labels shape:', labels.shape)
	print('ts_lengths 0:', np.sum(ts_lengths[0]))
	print('ts_lengths 1:', np.sum(ts_lengths[1]))
	return train_loader, test_loader, labels, ts_lengths, enc_feats

In [9]:
train_loader, test_loader, labels, ts_lengths, enc_feats = load_dataset2('UCR')
print(train_loader.shape, test_loader.shape)
true_labels = (np.sum(labels, axis=1) >= 1) + 0
print(true_labels.shape)
print(np.sum(true_labels), f'{np.sum(true_labels) / len(true_labels)*100:.2f} %')

anomaly_lens = consecutive_nonzero_lengths(true_labels)
print(anomaly_lens)
print(np.mean(anomaly_lens), np.max(anomaly_lens), np.min(anomaly_lens))

training set shape: (8100, 1)
test set shape: (21901, 1)
labels shape: (21901, 1)
ts_lengths 0: 8100
ts_lengths 1: 21901
(8100, 1) (21901, 1)
(21901,)
235 1.07 %
[12, 111, 102, 10]
58.75 111 10
